### TODO
1. [GCN Backprop](https://github.com/dmlc/dgl/issues/4021)
2. Visualize loss
3. [Embeddings](https://beta.openai.com/docs/guides/embeddings/what-are-embeddings)

In [1]:
import sys
import config
import argparse
import random
import networkx as nx
import numpy as np
from collections import defaultdict
from pyvis.network import Network
import matplotlib.pyplot as plt
from networkx.algorithms.community.modularity_max import greedy_modularity_communities

In [2]:
seed = 100
hidden_dim = 16
hidden_dim2 = 20
np.random.seed(seed)

### Graph data

In [3]:
G = nx.karate_club_graph()

In [4]:
G.number_of_nodes(), G.number_of_edges()

(34, 78)

### Generate labels from communities

In [5]:
communities = greedy_modularity_communities(G)
colors = np.zeros(G.number_of_nodes())
classes = set()

for i, c in enumerate(communities):
    colors[list(c)] = i
    classes.add(i)
    
num_classes = len(classes)
labels = (np.eye(len(classes))[colors.astype(int)]).T

classes, samples = labels.shape
print(f'classes: {classes}\t samples: {samples}')

classes: 3	 samples: 34


### Color nodes

In [6]:
def random_color():
    return '#%02X%02X%02X' % (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))

# uncomment for random colors
# color_map = {cls: random_color() for cls in classes}
color_map = {0: '#46FB47', 1: '#B9E6B5', 2: '#9F9EBF'}

colored_graph = Network(width='100%', notebook=True)

for node in G.nodes():
    colored_graph.add_node(node, color=color_map[int(colors[node])])
    
for edge in G.edges():
    colored_graph.add_edge(int(edge[0]), int(edge[1]))
    
colored_graph.show('colored_graph.html')

Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 


#### Renormalization trick

$A$ is the adjacency matrix, $I$ is the identity matrix, and $N$ is the cardinality of the set of nodes in the graph.

$$
\begin{align}
    \tilde{A} &= A + I_{N}\\
       \tilde{\mathcal{D}}_{ii} &= \sum_{i}\tilde{A}_{ij}\\
    \hat{\mathcal{A}}&=\tilde{\mathcal{D}}^{-\frac{1}{2}}\tilde{\mathcal{A}}\tilde{\mathcal{D}}^{-\frac{1}{2}}
\end{align}
$$

In [7]:
def renormalization(G):
    A = np.asarray(nx.to_numpy_matrix(G))
    I = np.eye(len(A))
    A_tilde = A + I
    D_tilde = np.zeros(A.shape, int)
    np.fill_diagonal(D_tilde, np.sum(A_tilde, axis=1).flatten())
    D_tilde = np.linalg.inv(D_tilde)
    D_tilde = np.power(D_tilde, 0.5)
    return D_tilde @ A_tilde @ D_tilde

In [8]:
nx.to_numpy_matrix(G)

matrix([[0., 4., 5., ..., 2., 0., 0.],
        [4., 0., 6., ..., 0., 0., 0.],
        [5., 6., 0., ..., 0., 2., 0.],
        ...,
        [2., 0., 0., ..., 0., 4., 4.],
        [0., 0., 2., ..., 4., 0., 5.],
        [0., 0., 0., ..., 4., 5., 0.]])

In [9]:
# Must pre-process offline
A_hat = renormalization(G)
print(A_hat)

[[0.02325581 0.11136921 0.13076645 ... 0.06502561 0.         0.        ]
 [0.11136921 0.03333333 0.18786729 ... 0.         0.         0.        ]
 [0.13076645 0.18786729 0.02941176 ... 0.         0.0549235  0.        ]
 ...
 [0.06502561 0.         0.         ... 0.04545455 0.13655775 0.12182898]
 [0.         0.         0.0549235  ... 0.13655775 0.02564103 0.11437725]
 [0.         0.         0.         ... 0.12182898 0.11437725 0.02040816]]


### Helper functions

In [10]:
def init(model, scheme):
    for i, layer in enumerate(model.parameters):
        model.parameters[i].W = scheme(*layer.W.shape)
        model.parameters[i].b = scheme(*layer.b.shape)
        
    return model

def glorot_init(in_dim, out_dim):
    sd = np.sqrt(6.0 / (in_dim + out_dim))
    return np.random.uniform(-sd, sd, size=(in_dim, out_dim))

In [11]:
class GradientDescent(object):
    def __init__(self, parameters, learning_rate):
        self.parameters = parameters
        self.learning_rate = learning_rate
        
        
    def zero_gradients(self):
        for layer in self.parameters:
            layer.dW = np.zeros(layer.W.shape)
            layer.db = np.zeros(layer.b.shape)
    
    
    def step(self):
        for i, layer in enumerate(self.parameters):
            # TODO: Replace with assertion
            if np.any(np.isnan(layer.db)):
                print(f'nans layer {i}')
                
            layer.W -= self.learning_rate * layer.dW
            layer.b -= self.learning_rate * layer.db

### Activation functions

In [12]:
def relu(x):
    return np.maximum(0, x)

def relu_(x):
    return (x > 0).astype(int)

def softmax(x, axis=0):
    return np.exp(x)/np.sum(np.exp(x), axis=axis, keepdims=True)

### Graph Convolutional Layer

$$
\text{ReLU}(\hat{A}XW^{1}+b^{1})
$$

In [13]:
class GCLayer(object):
    def __init__(self, input_dim, output_dim, name=''):
        self.name = name
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.W = glorot_init(output_dim, input_dim)
        self.dW = np.zeros(self.W.shape)
        self.b = np.ones((output_dim, 1))
        self.db = np.zeros(self.b.shape)
            
    '''
    inputs:
    G (nx.Graph)   Normalized Laplacian matrix for a static graph.
                   Dimensions: N x N where N is the number of nodes.
    x (np.ndarray) Embedding matrix
                   Dimensions: N x F where F is the number of features.
    '''
    def __call__(self, G, x, activation=None):
        if not activation:
            activation = lambda x: x
            
        # (nodes x nodes), (nodes x features), so need to transpose
        # before taking linear combination
        self.z = x # (n,f)
        
        # need to apply the activations along feature/hidden dimension
        # since x is (n,f), transpose to apply activations, then transpose back
        # to dot with the adjacency matrix
        self.a = activation(x.T).T
        
        # (n,n) x (n,f) -> (n,f).T -> (f,n) so can left-multiply weight with features
        # this is purely stylistic preference.
        X = (G @ self.a).T
        
        #print(f'({self.name}) W.shape: {self.W.shape}\t X.shape: {X.shape}')
        
        # transpose so can multiply by adjacency matrix in next layer, (n,h)
        return (self.W @ X + self.b).T # (h,f) x (f,n) + (h,1) -> (h,n). Broadcast bias vector.
    
    
    def backward(self, error, derivative=None):
        if not derivative:
            derivative = lambda x: x
            
        #print(f'({self.name}) W.T.shape: {self.W.T.shape}\t error.shape: {error.shape}\t a.shape: {self.a.shape}')
        
        #self.dW = error @ self.a.T # (h,n) x (n,f) -> (h,f) which matches W.shape
        self.dW = error @ self.a
        self.db = np.sum(error, axis=1, keepdims=True) # (h,n)
        
        return self.W.T @ error * derivative(self.z).T

    

### Linear Layer

In [14]:
class Linear(object):
    def __init__(self, input_dim, output_dim, name=''):
        self.name = name
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.W = glorot_init(output_dim, input_dim)
        self.dW = np.zeros(self.W.shape)
        self.b = np.ones((self.output_dim, 1))
        self.db = np.zeros(self.b.shape)    

    '''
    inputs:
    x (np.ndarray) Inputs to this layer
    
    outputs:
    a (np.ndarray) Output activations
    '''
    def __call__(self, x, activation=None):
        if not activation:
            activation = lambda x: x
            
        self.z = x # (f,n)
        self.a = activation(x)
        #print(f'({self.name}) W.shape: {self.W.shape}\t a.shape: {self.a.shape}')
        return self.W @ self.a + self.b # (h,f) x (f,n) + (h,1) -> (h,n). Broadcast bias vector.
    
    
    '''
    inputs:
    error (np.ndarray) Error signal of shape (W.out_dim, batch_size) from subsequent layer
    
    outputs:
    
    '''
    def backward(self, error, derivative=None):
        if not derivative:
            derivative = lambda x: x
            
        batch_size = error.shape[1]
        
        #print(f'({self.name}) W.T.shape: {self.W.T.shape}\t error.shape: {error.shape}\t a.T.shape: {self.a.T.shape}\t z.shape: {self.z.shape}')
        
        self.dW = error @ self.a.T # (h,n) x (n,f)        
        self.db = np.sum(error, axis=1, keepdims=True) # (h,n)
        
        return self.W.T @ error * derivative(self.z) # (f,h) x (h,n) * (h,n)
        

### Graph Convolutional Network

In [15]:
class GCN(object):
    def __init__(self, graph, num_classes):
        self.G = graph
        self.nodes = self.G.shape[0]
        self.embedding = np.eye(self.nodes)
        self.l0 = GCLayer(self.nodes, hidden_dim, name='l0')
        self.l1 = GCLayer(hidden_dim, hidden_dim2, name='l1')
        self.l2 = Linear(hidden_dim2, num_classes, name='l2')
        self.parameters = [self.l0, self.l1, self.l2]
        
    
    def __call__(self, x):
        a0 = self.l0(self.G, x, activation=relu)
        a1 = self.l1(self.G, a0, activation=relu).T # transpose b/c Linear layer expects (f,n)
        a2 = self.l2(a1)
        return softmax(a2)
    
    
    def backward(self, x):
        # Transpose errors from (34,3) -> (3,34) because linear weights are (16,34), but transposed for BP
        # so computation must be (labels, batch_size) x (batch_size, hidden_dim) to get error @ x.T
        d2 = self.l2.backward(x, derivative=relu_)
        d1 = self.l1.backward(d2, derivative=relu_)
        self.l0.backward(d1, derivative=None)
        

### Forward pass

#### Layer 1
$$
\begin{align}
    &\text{ReLU}
    \Biggl(
    \
        \underset{\mathcal{W}^{(1)}\ \in\ \mathbb{R}^{16\times34}}{
        \begin{bmatrix}
            w_{1,1} & \ldots & w_{1,34}\\
            \vdots & \ddots & \vdots\\
            w_{16,1} & \ldots & w_{16,34}
        \end{bmatrix}}
        \Biggl(
        \
            \underset{\hat{\mathcal{A}}\ \in\ \mathbb{R}^{34\times34}}{
            \begin{bmatrix}
                \alpha_{1,1} & \ldots & \alpha_{1,34}\\
                \vdots & \ddots & \vdots\\
                \alpha_{34,1} & \ldots & \alpha_{34,34}
            \end{bmatrix}}
            \ 
            \underset{\mathcal{X}\ \in\ \mathbb{R}^{34\times34}}{
            \begin{bmatrix}
                x_{1,1} & \ldots & x_{1,34}\\
                \vdots & \ddots & \vdots\\
                x_{34,1} & \ldots & x_{34,34}
            \end{bmatrix}}
        \
        \Biggl)^{\top}
        +
        \underset{\mathcal{b}^{(1)}\ \in\ \mathbb{R}^{16\times34}}{
        \begin{bmatrix}
            b_{1,1} & \ldots & b_{1,34}\\
            \vdots & \ddots & \vdots\\
            b_{16,1} & \ldots & b_{16,34}
        \end{bmatrix}}
    \
    \Biggr)\\
    =\
    &\text{ReLU}
    \Biggl(
    \
        \underset{\mathcal{Z}^{(1)}\ \in\ \mathbb{R}^{16\times34}}{
        \begin{bmatrix}
            z_{1,1} & \ldots & z_{1,34}\\
            \vdots & \ddots & \vdots\\
            z_{16,1} & \ldots & z_{16,34}
        \end{bmatrix}}
    \
    \Biggr)\\
    =
    &\quad\quad\quad\underset{\mathcal{A}^{(2)}\ \in\ \mathbb{R}^{16\times34}}{
    \begin{bmatrix}
        a_{1,1} & \ldots & a_{1,34}\\
        \vdots & \ddots & \vdots\\
        a_{16,1} & \ldots & a_{16,34}
    \end{bmatrix}}
    \rightarrow
    \Biggl(
    \
        \underset{\mathcal{A}^{(2)}\ \in\ \mathbb{R}^{16\times34}}{
        \begin{bmatrix}
            a_{1,1} & \ldots & a_{1,34}\\
            \vdots & \ddots & \vdots\\
            a_{16,1} & \ldots & a_{16,34}
        \end{bmatrix}}
    \
    \Biggl)^{\top}
\end{align}
$$

#### Layer 2
$$
\begin{align}
    &\text{ReLU}
    \Biggl(
    \
        \underset{\mathcal{W}^{(2)}\ \in\ \mathbb{R}^{16\times16}}{
        \begin{bmatrix}
            w_{1,1} & \ldots & w_{1,16}\\
            \vdots & \ddots & \vdots\\
            w_{16,1} & \ldots & w_{16,16}
        \end{bmatrix}}
        \
        \Biggl(
        \
            \underset{\hat{\mathcal{A}}\ \in\ \mathbb{R}^{34\times34}}{
            \begin{bmatrix}
                \alpha_{1,1} & \ldots & \alpha_{1,34}\\
                \vdots & \ddots & \vdots\\
                \alpha_{34,1} & \ldots & \alpha_{34,34}
            \end{bmatrix}}
            \
            \underset{\mathcal{A}^{(2)^{{\top}}}\ \in\ \mathbb{R}^{34\times16}}{
            \begin{bmatrix}
                a_{1,1} & \ldots & a_{1,16}\\
                \vdots & \ddots & \vdots\\
                a_{34,1} & \ldots & a_{34,16}
            \end{bmatrix}}
        \
        \Biggl)^{\top}
        +
        \underset{\mathcal{b}^{(2)}\ \in\ \mathbb{R}^{16\times34}}{
        \begin{bmatrix}
            b_{1,1} & \ldots & b_{1,34}\\
            \vdots & \ddots & \vdots\\
            b_{16,1} & \ldots & b_{16,34}
        \end{bmatrix}}
    \
    \Biggr)\\
    =\
    &\text{ReLU}
    \Biggl(
    \
        \underset{\mathcal{Z}^{(2)}\ \in\ \mathbb{R}^{16\times34}}{
        \begin{bmatrix}
            z_{1,1} & \ldots & z_{1,34}\\
            \vdots & \ddots & \vdots\\
            z_{16,1} & \ldots & z_{16,34}
        \end{bmatrix}}
    \
    \Biggr)\\
    =
    &\quad\quad\quad\underset{\mathcal{A}^{(3)}\ \in\ \mathbb{R}^{16\times34}}{
    \begin{bmatrix}
        a_{1,1} & \ldots & a_{1,34}\\
        \vdots & \ddots & \vdots\\
        a_{16,1} & \ldots & a_{16,34}
    \end{bmatrix}}
    \rightarrow
    \Biggl(
    \
        \underset{\mathcal{A}^{(3)}\ \in\ \mathbb{R}^{16\times34}}{
        \begin{bmatrix}
            a_{1,1} & \ldots & a_{1,34}\\
            \vdots & \ddots & \vdots\\
            a_{16,1} & \ldots & a_{16,34}
        \end{bmatrix}}
    \Biggl)^{\top}
\end{align}
$$


#### Layer 3
$$
\begin{align}
    &\text{Softmax}
    \Biggl(
    \
        \underset{\mathcal{W}^{(3)}\ \in\ \mathbb{R}^{3\times16}}{
        \begin{bmatrix}
            w_{1,1} & \ldots & w_{1,16}\\
            w_{2,1} & \ldots & w_{2,16}\\
            w_{3,1} & \ldots & w_{3,16}
        \end{bmatrix}}
        \
        \Biggl(
        \
            \underset{\hat{\mathcal{A}}\ \in\ \mathbb{R}^{34\times34}}{
            \begin{bmatrix}
                \alpha_{1,1} & \ldots & \alpha_{1,34}\\
                \vdots & \ddots & \vdots\\
                \alpha_{34,1} & \ldots & \alpha_{34,34}
            \end{bmatrix}}
            \ 
            \underset{\mathcal{A}^{(3)^{\top}}\ \in\ \mathbb{R}^{34\times16}}{
            \begin{bmatrix}
                a_{1,1} & \ldots & a_{1,16}\\
                \vdots & \ddots & \vdots\\
                a_{34,1} & \ldots & a_{34,16}
            \end{bmatrix}}
        \Biggl)^{\top}
        +
        \underset{\mathcal{b}^{(3)}\ \in\ \mathbb{R}^{3\times34}}{
        \begin{bmatrix}
            b_{1,1} & \ldots & b_{1,34}\\
            b_{2,1} & \ldots & b_{2,34}\\
            b_{3,1} & \ldots & b_{3,34}
        \end{bmatrix}}
    \
    \Biggr)\\
    =\
    &\text{Softmax}
    \Biggl(
    \
        \underset{\mathcal{Z}^{(3)}\ \in\ \mathbb{R}^{3\times34}}{
        \begin{bmatrix}
            z_{1,1} & \ldots & z_{1,34}\\
            z_{2,1} & \ldots & z_{2,34}\\
            z_{3,1} & \ldots & z_{3,34}
        \end{bmatrix}}
    \
    \Biggr)\\
    =
    &\quad\quad\quad\quad\underset{\mathcal{A}^{(4)}\ \in\ \mathbb{R}^{3\times34}}{
    \begin{bmatrix}
        a_{1,1} & \ldots & a_{1,34}\\
        a_{2,1} & \ldots & a_{2,34}\\
        a_{3,1} & \ldots & a_{3,34}
    \end{bmatrix}}
\end{align}
$$


### Loss function

Why we use cross entropy loss for classification when doing MLE:
https://en.wikipedia.org/wiki/Cross_entropy#Relation_to_maximum_likelihood

In [16]:
def cross_ent(predictions, targets):
    N = predictions.shape[1] # (3,34), so index 1 for samples
    targets_ = np.squeeze(np.asarray(targets))
    predictions_ = np.squeeze(np.asarray(predictions))
    ce = -np.sum(targets_*np.log(predictions_))/N
    return ce

### Backpropagation

#### Cross entropy loss

$$
\begin{align}
    \delta^{(4)}=&\quad\frac{\partial}{\partial z^{(3)}}\ \frac{1}{2} \Big\lVert Y-H_{\mathcal{W},\mathcal{b}}(\mathcal{X})\Big\rVert^{2}\\
    =&\quad\mathcal{A}^{(4)}-Y\\
    =&\underset{\mathcal{A}^{(4)}\ \in\ \mathbb{R}^{3\times34}}{
    \begin{bmatrix}
        a_{1,1} & \ldots & a_{1,34}\\
        a_{2,1} & \ldots & a_{2,34}\\
        a_{3,1} & \ldots & a_{3,34}
    \end{bmatrix}}
    -
    \underset{\mathcal{Y}^{(4)}\ \in\ \mathbb{R}^{3\times34}}{
    \begin{bmatrix}
        y_{1,1} & \ldots & y_{1,34}\\
        y_{2,1} & \ldots & y_{2,34}\\
        y_{3,1} & \ldots & y_{3,34}
    \end{bmatrix}}\\
    =&\underset{\mathcal{\delta}^{(4)}\ \in\ \mathbb{R}^{3\times34}}{
    \begin{bmatrix}
        d_{1,1} & \ldots & d_{1,34}\\
        d_{2,1} & \ldots & d_{2,34}\\
        d_{3,1} & \ldots & d_{3,34}
    \end{bmatrix}}
\end{align}
$$

#### Layer 3
$$
\begin{align}
    \nabla W^{(3)} =& \delta^{(4)}A^{(3)^{\top}}\\
    =& 
    \underset{\mathcal{\delta}^{(4)}\ \in\ \mathbb{R}^{3\times34}}{
    \begin{bmatrix}
        d_{1,1} & \ldots & d_{1,34}\\
        d_{2,1} & \ldots & d_{2,34}\\
        d_{3,1} & \ldots & d_{3,34}
    \end{bmatrix}}
    \
    \underset{\mathcal{A}^{(3)^{\top}}\ \in\ \mathbb{R}^{34\times16}}{
    \begin{bmatrix}
        a_{1,1} & \ldots & a_{1,16}\\
        \vdots & \ddots & \vdots\\
        a_{34,1} & \ldots & a_{34,16}
    \end{bmatrix}}\\
    =&
    \underset{\nabla\mathcal{W}^{(3)}\ \in\ \mathbb{R}^{3\times16}}{
    \begin{bmatrix}
        w_{1,1} & \ldots & w_{1,16}\\
        w_{2,1} & \ldots & w_{2,16}\\
        w_{3,1} & \ldots & w_{3,16}
    \end{bmatrix}}\\
    \nabla b^{(3)}=&\delta^{(4)}\\
    =&\underset{\mathcal{\delta}^{(4)}\ \in\ \mathbb{R}^{3\times34}}{
    \begin{bmatrix}
        d_{1,1} & \ldots & d_{1,34}\\
        d_{2,1} & \ldots & d_{2,34}\\
        d_{3,1} & \ldots & d_{3,34}
    \end{bmatrix}}\\
    \delta^{(3)} =&\mathcal{W}^{(3)^{\top}}\delta^{(4)}\odot\frac{\partial}{\partial z^{(2)}}\text{ReLU}(z^{(2)})\\
    =&
    \underset{\mathcal{W}^{(3)^{\top}}\ \in\ \mathbb{R}^{16\times3}}{
    \begin{bmatrix}
        w_{1,1} & w_{1,2} & w_{1,3}\\
        \vdots & \ddots & \vdots\\
        w_{16,1} & w_{16,2} & w_{16,3}
    \end{bmatrix}}
    \
    \underset{\mathcal{\delta}^{(4)}\ \in\ \mathbb{R}^{3\times34}}{
    \begin{bmatrix}
        d_{1,1} & \ldots & d_{1,34}\\
        d_{2,1} & \ldots & d_{2,34}\\
        d_{3,1} & \ldots & d_{3,34}
    \end{bmatrix}}
    \odot
    \frac{\partial}{\partial z^{(2)}}
    \text{ReLU}
    \Biggl(
    \ 
        \underset{\mathcal{Z}^{(2)}\ \in\ \mathbb{R}^{16\times34}}{
        \begin{bmatrix}
            z_{1,1} & \ldots & z_{1,34}\\
            \vdots & \ddots & \vdots\\
            z_{16,1} & \ldots & z_{16,34}
        \end{bmatrix}}
    \ \Biggl)\\
    =&
    \underset{\mathcal{\delta}^{(3)}\ \in\ \mathbb{R}^{16\times34}}{
    \begin{bmatrix}
        d_{1,1} & \ldots & d_{1,34}\\
        \vdots & \ddots & \vdots\\
        d_{16,1} & \ldots & d_{16,34}
    \end{bmatrix}}
\end{align}
$$

#### Layer 2
$$
\begin{align}
    \nabla W^{(2)} =& \delta^{(3)}A^{(2)^{\top}}\\
    =&
    \underset{\mathcal{\delta}^{(3)}\ \in\ \mathbb{R}^{16\times34}}{
    \begin{bmatrix}
        d_{1,1} & \ldots & d_{1,34}\\
        \vdots & \ddots & \vdots\\
        d_{16,1} & \ldots & d_{16,34}
    \end{bmatrix}}
    \
    \underset{\mathcal{A}^{(2)^{{\top}}}\ \in\ \mathbb{R}^{34\times16}}{
            \begin{bmatrix}
                a_{1,1} & \ldots & a_{1,16}\\
                \vdots & \ddots & \vdots\\
                a_{34,1} & \ldots & a_{34,16}
            \end{bmatrix}}\\
    \nabla b^{(2)} =& \delta^{(3)}\\
    =&
    \underset{\mathcal{\delta}^{(3)}\ \in\ \mathbb{R}^{16\times34}}{
    \begin{bmatrix}
        d_{1,1} & \ldots & d_{1,34}\\
        \vdots & \ddots & \vdots\\
        d_{16,1} & \ldots & d_{16,34}
    \end{bmatrix}}\\
    \delta^{(2)} =&\mathcal{W}^{(2)^{\top}}\delta^{(3)}\odot\frac{\partial}{\partial z^{(1)}}\text{ReLU}(z^{(1)})\\
    =&
    \underset{\mathcal{W}^{(2)^{\top}}\ \in\ \mathbb{R}^{16\times16}}{
    \begin{bmatrix}
        w_{1,1} & \ldots & w_{1,16}\\
        \vdots & \ddots & \vdots\\
        w_{16,1} & \ldots & w_{16,16}
    \end{bmatrix}}
    \
    \underset{\mathcal{\delta}^{(3)}\ \in\ \mathbb{R}^{16\times34}}{
    \begin{bmatrix}
        d_{1,1} & \ldots & d_{1,34}\\
        \vdots & \ddots & \vdots\\
        d_{16,1} & \ldots & d_{16,34}
    \end{bmatrix}}
    \
    \odot\frac{\partial}{\partial z^{(1)}}\text{ReLU}
    \Biggl(
    \
        \underset{\mathcal{Z}^{(1)}\ \in\ \mathbb{R}^{16\times34}}{
        \begin{bmatrix}
            z_{1,1} & \ldots & z_{1,34}\\
            \vdots & \ddots & \vdots\\
            z_{16,1} & \ldots & z_{16,34}
        \end{bmatrix}}
    \
    \Biggl)\\
    =&
    \underset{\mathcal{\delta}^{(2)}\ \in\ \mathbb{R}^{16\times34}}{
    \begin{bmatrix}
        d_{1,1} & \ldots & d_{1,34}\\
        \vdots & \ddots & \vdots\\
        d_{16,1} & \ldots & d_{16,34}
    \end{bmatrix}}
\end{align}
$$

#### Layer 1
$$
\begin{align}
    \nabla W^{(1)} =& \delta^{(2)}\mathcal{X}^{\top}\\
    =&
    \underset{\mathcal{\delta}^{(2)}\ \in\ \mathbb{R}^{16\times34}}{
    \begin{bmatrix}
        d_{1,1} & \ldots & d_{1,34}\\
        \vdots & \ddots & \vdots\\
        d_{16,1} & \ldots & d_{16,34}
    \end{bmatrix}}
    \ 
    \underset{\mathcal{X^{\top}}\ \in\ \mathbb{R}^{34\times34}}{
            \begin{bmatrix}
                x_{1,1} & \ldots & x_{1,34}\\
                \vdots & \ddots & \vdots\\
                x_{34,1} & \ldots & x_{34,34}
            \end{bmatrix}}\\
    \nabla b^{(1)} =& \delta^{(2)}\\
    =& \underset{\mathcal{\delta}^{(2)}\ \in\ \mathbb{R}^{16\times34}}{
    \begin{bmatrix}
        d_{1,1} & \ldots & d_{1,34}\\
        \vdots & \ddots & \vdots\\
        d_{16,1} & \ldots & d_{16,34}
    \end{bmatrix}}
\end{align}
$$

### Training loop

In [17]:
def train(model, loss, epochs, features, labels, opt):
    for e in range(epochs):
        output = model(features)
        loss_val = loss(output, labels)
        deriv_loss = output - labels
        
#         try:
        model.backward(deriv_loss)
        opt.step()
        opt.zero_gradients()
#         except:
#             print(f'error at epoch {e}')
#             sys.exit(0)
        
        if e % 100 == 0:
            print(f'(epoch {e}) loss: {loss_val}')

### Hyperparameters

In [18]:
epochs = 10000
lr = 0.001

In [19]:
features = np.eye(G.number_of_nodes())
model = GCN(A_hat, num_classes)
opt = GradientDescent(model.parameters, lr)
train(model, cross_ent, epochs, features, labels, opt)

loss: 2.567373113054218
epoch 0 loss: 2.567373113054218
loss: 1.5426103777974707
loss: 1.3519693398754289
loss: 1.223366740692084
loss: 1.127410506947819
loss: 1.0664283685789915
loss: 1.0364800052928775
loss: 1.0244247847866381
loss: 1.0198563145420791
loss: 1.0178217509658367
loss: 1.0165535727921744
loss: 1.0155052588310678
loss: 1.0145151519110935
loss: 1.0135366387473557
loss: 1.012545152076279
loss: 1.0115633969578737
loss: 1.0105824553202192
loss: 1.009608943674775
loss: 1.008634963421475
loss: 1.0076627824518964
loss: 1.0066921368323707
loss: 1.005731248285342
loss: 1.004769689814484
loss: 1.0038086486726547
loss: 1.0028474356625807
loss: 1.001887360378827
loss: 1.0009282663305354
loss: 0.9999699896179225
loss: 0.9990123696285794
loss: 0.9980552493988192
loss: 0.9970984748879634
loss: 0.9961418944159204
loss: 0.9951853583166096
loss: 0.994228718731259
loss: 0.9932718294823559
loss: 0.9923145459926862
loss: 0.9913557472936674
loss: 0.9903965329739907
loss: 0.9894366595573667
los

loss: 0.47243198713893575
loss: 0.4707840806282802
loss: 0.46913319265219844
loss: 0.46750447072685747
loss: 0.4658779336742775
loss: 0.4642525828408585
loss: 0.4626483721186596
loss: 0.4610387059332496
loss: 0.4594507774055386
loss: 0.45786481037288473
loss: 0.4562794803234239
loss: 0.45470923368418115
loss: 0.45313843510169804
loss: 0.4515834844928766
loss: 0.4500284175290331
loss: 0.4484892025538442
loss: 0.44694971335146255
loss: 0.44542585968869935
loss: 0.44390835409965457
loss: 0.4423900358429136
loss: 0.4408881290264388
loss: 0.43938556208701646
loss: 0.4378987652074679
loss: 0.4364109775178938
loss: 0.4349387514715237
loss: 0.4334670434273005
loss: 0.43200920347304167
loss: 0.4305496747974809
loss: 0.4291007470454695
loss: 0.4276446973122763
loss: 0.4262061961173832
loss: 0.4247699083280031
loss: 0.4233578104056692
loss: 0.42193008828194856
loss: 0.42051882692418247
loss: 0.41910946990784254
loss: 0.41771282950669314
loss: 0.416332863453738
loss: 0.4149410263058747
loss: 0.413

loss: 0.10355899401907796
loss: 0.10328813904405798
loss: 0.10301814485919836
loss: 0.10274900961691831
loss: 0.10248073086454795
loss: 0.10221330574251777
loss: 0.10194673111623828
loss: 0.10168100366473304
loss: 0.10140732468891418
loss: 0.10114555355878325
loss: 0.10088385968879776
loss: 0.10062250007685003
loss: 0.10036164130860555
loss: 0.10010139079735503
loss: 0.09984181746027747
loss: 0.0995745176681802
loss: 0.09931858361750308
loss: 0.0990601218162088
loss: 0.09880271160552413
loss: 0.098546251589613
loss: 0.09828252775568468
loss: 0.09802985847499347
loss: 0.0977682438822164
loss: 0.09750945127479496
loss: 0.09725278560214849
loss: 0.0969970320041859
loss: 0.09674274326356784
loss: 0.0964897421910066
loss: 0.09623790074779866
loss: 0.09598712497665318
loss: 0.09573734488575726
loss: 0.09548850756131919
loss: 0.09524057241538077
loss: 0.0949935078644883
loss: 0.09474728897907371
loss: 0.09450189579932257
loss: 0.09425731211413231
loss: 0.09401077888813876
loss: 0.093765716018

loss: 0.040145815681940644
loss: 0.04005165986596296
loss: 0.03995770583030961
loss: 0.0398606200427781
loss: 0.0397678760580724
loss: 0.039675186353901264
loss: 0.03958259565855099
loss: 0.03949013879094504
loss: 0.03939784269556069
loss: 0.03930572811699387
loss: 0.039213810950946996
loss: 0.0391221033193851
loss: 0.039027422233215044
loss: 0.03893689279900112
loss: 0.03884642820013029
loss: 0.03875606915925891
loss: 0.038665847440151786
loss: 0.038575787676759885
loss: 0.03848590887806527
loss: 0.03839622564334675
loss: 0.038303692942941
loss: 0.038215130711887424
loss: 0.038126633508066436
loss: 0.03803824053118872
loss: 0.03794998244859685
loss: 0.037861883114304905
loss: 0.03777396098939273
loss: 0.037686230293612344
loss: 0.03759870192686364
loss: 0.037508469687563196
loss: 0.03742203272335446
loss: 0.037335670047787387
loss: 0.037249417292839405
loss: 0.037163302388260135
loss: 0.03707734710429156
loss: 0.036991568326092523
loss: 0.03690597908797593
loss: 0.03681780019859929
lo

loss: 0.016530359998101176
loss: 0.016501190098289157
loss: 0.016472090129337823
loss: 0.01644306072814005
loss: 0.016414102413713074
loss: 0.016385215602727238
loss: 0.016356400623147648
loss: 0.016327657726178803
loss: 0.01629898709669185
loss: 0.016270388862307167
loss: 0.01624158035531238
loss: 0.01621318190973325
loss: 0.016184848377103662
loss: 0.016156580740766823
loss: 0.01612837982644044
loss: 0.01610024632164327
loss: 0.016072180793072155
loss: 0.016044183702048243
loss: 0.016016255418177656
loss: 0.01598839623138312
loss: 0.015960606362465022
loss: 0.015932885972344855
loss: 0.015905235170137505
loss: 0.015877392840397016
loss: 0.015849932047478577
loss: 0.015822534002959764
loss: 0.015795199581754404
loss: 0.015767929518725066
loss: 0.015740724425668474
loss: 0.01571358480654311
loss: 0.01568651107103607
loss: 0.015659503546590483
loss: 0.01563256248902474
loss: 0.015605688091876738
loss: 0.015578880494604005
loss: 0.01555213292541445
loss: 0.015525452317201881
loss: 0.0154

...

loss: 0.0007138776668756501
loss: 0.0007137476981657877
loss: 0.0007136177653714193
loss: 0.0007134878685637172
loss: 0.000713358007812854
loss: 0.0007132281831878975
loss: 0.0007130983947569164
loss: 0.0007129686425869343
loss: 0.0007128389267440046
loss: 0.0007126975455426152
loss: 0.0007125756850389545
loss: 0.0007124461199150183
loss: 0.0007123165908895605
loss: 0.0007121870980293612
loss: 0.0007120576413999909
loss: 0.0007119282210661946
loss: 0.0007117988370915683
loss: 0.0007116578080605463
loss: 0.0007115286161438753
loss: 0.0007114071171766372
loss: 0.000711277919088842
loss: 0.0007111487571992519
loss: 0.0007110196315712352
loss: 0.0007108905422671258
loss: 0.0007107614893484379
loss: 0.0007106208116943685
loss: 0.0007104919505653466
loss: 0.0007103631246540271
loss: 0.000710234334033269
loss: 0.0007101055787749422
loss: 0.000709976858949815
loss: 0.0007098481746276144
loss: 0.0007097271778319504
loss: 0.0007095868470101335
loss: 0.0007094583125029354
loss: 0.0007093298132693

loss: 0.0006610035465134496
loss: 0.0006608906365190546
loss: 0.0006607777560160054
loss: 0.0006606649050559289
loss: 0.0006605520836898293
loss: 0.000660439291967872
loss: 0.0006603265299395801
loss: 0.0006602137976536732
loss: 0.0006600905142684073
loss: 0.0006599852518075946
loss: 0.0006598726408833704
loss: 0.0006597600595380564
loss: 0.0006596475078208882
loss: 0.0006595349857802705
epoch 8400 loss: 0.0006595349857802705
loss: 0.0006594224934640533
loss: 0.0006593100309192612
loss: 0.0006591975981922965
loss: 0.000659074634372652
loss: 0.0006589696656569192
loss: 0.0006588573543394275
loss: 0.0006587450726690978
loss: 0.0006586328206928847
loss: 0.0006585205984569438
loss: 0.0006584084060068904
loss: 0.0006582962433875789
loss: 0.0006581735679609094
loss: 0.000658061566747236
loss: 0.0006579495944082557
loss: 0.0006578376509978889
loss: 0.0006577257365692569
loss: 0.0006576211471153491
loss: 0.0006575092218401727
loss: 0.0006573973264290535
loss: 0.0006572749362785243
loss: 0.0006

loss: 0.0006174973107981134
loss: 0.0006173974887718826
loss: 0.0006172976928091737
loss: 0.0006171882528156892
loss: 0.0006170955929553582
loss: 0.000616995900283284
epoch 8800 loss: 0.000616995900283284
loss: 0.0006168962335136039
loss: 0.0006167965926734819
loss: 0.000616696977789827
loss: 0.0006165973888890329
loss: 0.0006164978259970887
loss: 0.0006163982891396345
loss: 0.0006162987783418338
loss: 0.0006161896439559462
loss: 0.0006160972620209253
loss: 0.0006159978543460469
loss: 0.000615898472566602
loss: 0.0006157991167086714
loss: 0.0006156997867978129
loss: 0.0006156004828592888
loss: 0.000615501204917933
loss: 0.0006154019529982244
loss: 0.0006152930962007341
loss: 0.0006152009662396218
loss: 0.000615101817189001
loss: 0.0006150028409538755
loss: 0.0006149038894555282
loss: 0.0006148049627287887
loss: 0.0006147060608079475
loss: 0.0006146071837269397
loss: 0.0006145083315191029
loss: 0.0006144095042174097
loss: 0.0006143010912899186
loss: 0.0006142024020644404
loss: 0.0006141

loss: 0.0005766797379397629
loss: 0.000576591560876603
loss: 0.0005765034066244539
loss: 0.0005764152751932393
loss: 0.0005763271665926342
loss: 0.0005762390808321647
loss: 0.0005761510179211897
loss: 0.0005760629778688607
loss: 0.0005759749606840606
loss: 0.0005758936436482611
loss: 0.0005757968204123624
loss: 0.0005757088921230153
loss: 0.0005756209865317353
loss: 0.0005755331036486763
loss: 0.0005754452434836146
loss: 0.000575357406046313
loss: 0.0005752695913462225
loss: 0.0005751817993926067
loss: 0.0005750940301946658
loss: 0.0005750129531447482
loss: 0.0005749163970602922
loss: 0.0005748287162970919
loss: 0.000574741058122101
loss: 0.0005746534225455818
loss: 0.0005745658095774685
loss: 0.0005744782192276029
loss: 0.0005743906515056162
loss: 0.0005743031064208709
loss: 0.0005742155839826594
loss: 0.0005741347457255674
loss: 0.0005740384554867282
loss: 0.0005739510210319436
loss: 0.0005738636090582454
loss: 0.0005737762195760011
loss: 0.000573688852595302
loss: 0.0005736015081260

loss: 0.0005428243443730864
loss: 0.0005427452918524388
loss: 0.0005426726486245925
loss: 0.0005425854939249612
loss: 0.0005425065188615787
loss: 0.0005424275641865925
loss: 0.0005423486298989259
loss: 0.0005422697159974397
loss: 0.0005421908224809715
loss: 0.0005421119493482176
loss: 0.0005420330965979299
loss: 0.0005419542642286309
loss: 0.0005418818326799929
loss: 0.0005418029885499474
loss: 0.0005417160941694696
loss: 0.000541637359483715
loss: 0.0005415586450152455
loss: 0.0005414799507638242
loss: 0.0005414012767290868
loss: 0.0005413226229106221
loss: 0.00054124398930797
loss: 0.000541171748755823
loss: 0.0005410931031854867
loss: 0.000541014478310337
loss: 0.0005409358741250559
loss: 0.0005408572906244205
loss: 0.0005407787278030701
loss: 0.0005406921372371402
loss: 0.0005406136831968935
loss: 0.0005405352491938862
loss: 0.000540463200324557
loss: 0.0005403847541225778
loss: 0.0005403063284384555
loss: 0.000540227923267926
loss: 0.0005401495386067445
loss: 0.0005400711744506494